In [1]:
from __future__ import print_function # In Python2, print is a statement, in Python3, print is a function

In [2]:
import requests

# 1. Google Trend(Char only)
https://www.google.com.tw/trends/fetchComponent?q=%2Fm%2F016c3c&date=now%207-d&cid=TIMESERIES_GRAPH_0&export=5&w=0&h=0&nhf=1
https://www.google.com.tw/trends/explore#q=%E4%B9%9D%E4%BB%BD

### 將資料從Google Trend中抓下 

In [3]:
from bs4 import BeautifulSoup
import json
import time

res = requests.get('https://www.google.com.tw/trends/trendsReport?hl=zh-TW&q=%E4%B9%9D%E4%BB%BD&tz=Etc%2FGMT-8&content=1')
soup = BeautifulSoup(res.text,'html5lib')
res.encoding = 'utf-8'

datas = soup.select('#TIMESERIES_GRAPH_0')[0]  #抓id='TIMESERIES_GRAPH_0'的內容
# print(type(datas))  #<class 'bs4.element.Tag'>
# print(datas)

time.sleep(3)  #避免被擋掉，小睡一會兒

data_sc = [s.extract() for s in datas.select('script')][0].text #只留<script>裡的資料

#strip()將特殊字或空白去除;split(" = ")[1]剖析並取出需要的內容=><type 'unicode'>
datalist = data_sc.strip().split(" = ")[1].split('"rows":[')[1].split('],"showHeadlines":')[0].replace('],[', ']+[').split("+")
print(len(datalist))
print(type(datalist))
print(datalist[0])

data_total_list = []   #建立陣列
#for i in range(0,len(datalist)):
    #data_total_list.append(datalist[i])
#print(data_total_list)


150
<type 'list'>
[{"v":new Date(2004, 0, 16, 20, 0),"f":"2004年1月"},null,null,57,null,null,true]


In [4]:
datalist[0]

u'[{"v":new Date(2004, 0, 16, 20, 0),"f":"2004\u5e741\u6708"},null,null,57,null,null,true]'

### 將抓下的資料剖析，只留下年、月、數值，且以Json格式呈現

In [39]:
# 因為我們要parse的是Javascript code，所以他沒辦法被轉換為Python的data structure，
# 這邊我們只能透過regex（正規表達式）的方式來做字串的extract。
import re

res_list = []
for data in datalist:
#     print(data)
    #抓時間點的搜尋數值
    res_dict = {}
    res = re.search(',([0-9]+),', data) # 搜尋包含在兩個,之中的任何長度數字=>從data中找到 ",(任一數字),"
#     print(res.group(1)) # group(1) 表示我們()內的capture group
    
    #抓時間點
    pat='([0-9]+)年([0-9]+)月'.decode("utf8") # 設定我們抓取中文的regex，注意這邊要先把str變成unicode
    res2 = re.search(pat, data)
    #print(res2)
    res_dict['year'] = res2.group(1)  #定義key名稱='year'
    res_dict['month'] = res2.group(2) #定義key名稱='month'
    res_dict['value'] = res.group(1)  #定義key名稱='value'
    res_list.append(res_dict)
print(res_list[1])

{'month': u'2', 'value': u'64', 'year': u'2004'}


### 將Json格式資料寫到指定路徑之檔案，檔案結尾為今日日期

In [57]:
# print(type(res_list[0]))  #dict
# print(len(res_list)) #150
import datetime
today = datetime.date.today()
# print(today)

fid = open("C:\Users\BIG DATA\Desktop\google_trend{}.txt".format(today),"w")

for one in res_list:
    fid.write(str(one.items())) 

fid.close()

2016-06-29


In [56]:
import datetime
today = datetime.date.today()
print(today)

2016-06-29
